In [59]:
import pandas as pd
import numpy as np

In [72]:
actual = r"D:\Wild deserts photos\concatenated_Dave.xlsx"

In [73]:
df = pd.read_excel(actual, dtype=str)
df['Time'] = pd.to_timedelta(df['Time']) 
df['Date'] = pd.to_datetime(df['Date'])  
df["datetime_converted"] = df["Date"]+df["Time"]

In [64]:
from pathlib import Path
import pandas as pd

root_dir = Path(r"D:\Wild deserts photos\2024_12_WCAM_originals")

all_dataframes = []

# Recursively search for all .csv files under root_dir
for csv_file in root_dir.rglob("*.csv"):
    # Read the CSV into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Extract information from the parent directory name
    parent_dir = csv_file.parent.name
    
    # Create a new column based on the directory name
    df["model_name"] = parent_dir
    
    # If you want the entire path (relative to root_dir), you could do:
    relative_path = csv_file.relative_to(root_dir)  # e.g., "subfolder/another_subfolder/data.csv"
    # df["relative_path"] = str(relative_path.parent)  # e.g., "subfolder/another_subfolder"

    # If you need multiple levels of folders in separate columns, for example:
    level_one_folder = relative_path.parts[0]  
    df["Location"] = level_one_folder

    # Store the modified DataFrame
    all_dataframes.append(df)

# Concatenate all DataFrames into one (optional)
if all_dataframes:
    master_df = pd.concat(all_dataframes, ignore_index=True)
    print(master_df)
    # Optionally save to a new CSV
    # master_df.to_csv("combined_data.csv", index=False)
else:
    print("No CSV files found.")


                                           path_original   species confidence                                               bbox  \
0      E:\Wild deserts photos\2024_12_WCAM_originals/...      none       none                                               none   
1      E:\Wild deserts photos\2024_12_WCAM_originals/...      none       none                                               none   
2      E:\Wild deserts photos\2024_12_WCAM_originals/...      none       none                                               none   
3      E:\Wild deserts photos\2024_12_WCAM_originals/...      none       none                                               none   
4      E:\Wild deserts photos\2024_12_WCAM_originals/...      none       none                                               none   
...                                                  ...       ...        ...                                                ...   
62606  E:\Wild deserts photos\2024_12_WCAM_originals/...    Rabbit      0.96

In [65]:
master_df["datetime_converted"] = pd.to_datetime(master_df["time"], format="%Y:%m:%d %H:%M:%S")
master_df["Date"] = master_df["datetime_converted"].dt.date
master_df["Time"] = master_df["datetime_converted"].dt.time


In [91]:
df = df[
    (df["datetime_converted"] > master_df["datetime_converted"].min()) &
    (df["datetime_converted"] < master_df["datetime_converted"].max())
]


In [103]:
joined_df = pd.merge(master_df, df, on = "datetime_converted", how = "right")

In [106]:
cleaned = joined_df[[
    "species", 
    "confidence",
    "Species", 
    "path_original", 
    "datetime_converted", 
    "Location_x", 
    "Location_y", 
    "model_name"
]]
cleaned = cleaned.replace("Red Kangaroo", "Kangaroo")

In [107]:
not_correct = cleaned.loc[~(cleaned["Species"]==cleaned["species"])]

In [110]:
night = not_correct.loc[not_correct["model_name"]=="night_0402"]